In [1]:
import pso
import particle
import os
from utils import load_cloud_dataset, load_breast_cancer, load_seg_data, load_digits_dataset, load_satelite_dataset, load_synthetic_dataset
import importlib
import numpy as np
import utils


In [45]:
importlib.reload(particle)
importlib.reload(pso)
importlib.reload(utils)

<module 'utils' from '/home/skholkin/projects/PSO_GMM/utils.py'>

In [46]:
config = pso.PSOConfig.from_json(os.path.join('configs', 'default_params_T1_T2_play.json'))

data = load_seg_data()
# synth_dataset_name = 'Synthetic_dim_50_n_samples_1000_n_comp_15_c_separation_1.0.data.npy'
# data = load_synthetic_dataset(synth_dataset_name)
n_runs = 10
eig_coef_list = [0.03, 0.1, 0.3, 1, 3]
eig_coef_list = [0.03]

for coef in eig_coef_list:
    config.eigvals_coef = coef

    res = []
    for i in range(n_runs):

        pso_algo = pso.PSOEigen(data, config)
        res.append(pso_algo.run())
        print(res[-1])

    for key in res[0].keys():
        value = np.mean([item[key] for item in res])

        std = np.std([item[key] for item in res])
        print('Eig Coef', coef)
        print(key, ': ', value, ' +- ', std)

Time for GMM init:  2.097351312637329  sec
{'em': 70.65717890491878, 'pso': 71.53164026031479, 'transformed': 73.19829176960612, 'ref_em': 71.05429635516167}
Time for GMM init:  1.5744094848632812  sec
{'em': 70.47482687759567, 'pso': 72.18043343305963, 'transformed': 72.6819568996033, 'ref_em': 70.60137880417805}
Time for GMM init:  1.7467379570007324  sec
{'em': 71.59198644836933, 'pso': 71.99320018879624, 'transformed': 73.90728105744076, 'ref_em': 70.1714827838533}
Time for GMM init:  1.5350518226623535  sec
{'em': 70.05098640922306, 'pso': 71.35932843648413, 'transformed': 72.58141198692773, 'ref_em': 70.34366812665263}
Time for GMM init:  1.6144742965698242  sec
{'em': 70.61952306546526, 'pso': 71.81520221287654, 'transformed': 72.70385043198343, 'ref_em': 70.28563411227556}
Time for GMM init:  1.5786943435668945  sec
{'em': 69.93237902120171, 'pso': 71.54252490630068, 'transformed': 73.0180862821541, 'ref_em': 70.69812965337286}
Time for GMM init:  1.6926374435424805  sec
{'em':

In [60]:
res_dict

{'em': 74.24607799008619,
 'pso': 74.48534941906306,
 'ref_em_score': 74.16039295495584}

In [1]:
import numpy as np
from utils import _givens_rotation_matrix_entries

def QRGivens_slow(A):
    if np.linalg.det(A) < 0:
        A = -A
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)
    phi_list = []

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
    i = 0
    for (row, col) in zip(rows, cols):
        i += 1

        # Compute Givens rotation matrix and
        # zero-out lower triangular matrix entries.
        (c, s) = _givens_rotation_matrix_entries(R[col, col], R[row, col])


        phi = np.arccos(c)
        # if sin(phi) < 0
        if s > 0:
            phi = -phi

        # Turning first element into 1 instead of -1
        if c * R[col, col] - s * R[row, col] < 0:
            phi = phi - np.pi
            c = -c
            s = -s

        G = np.identity(num_rows)
        G[col, col] = c
        G[row, row] = c
        phi_list.append(phi)
        G[row, col] = s
        G[col, row] = -s

        R = np.dot(G, R)

        Q = np.dot(Q, G.T)

    return Q, R, phi_list


def QRGivens_fast(A):
    if np.linalg.det(A) < 0:
        A = -A
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)
    phi_list = []

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
    i = 0
    for (row, col) in zip(rows, cols):
        i += 1

        # Compute Givens rotation matrix and
        # zero-out lower triangular matrix entries.
        (c, s) = _givens_rotation_matrix_entries(R[col, col], R[row, col])


        phi = np.arccos(c)
        # if sin(phi) < 0
        if s > 0:
            phi = -phi

        # Turning first element into 1 instead of -1
        if c * R[col, col] - s * R[row, col] < 0:
            phi = phi - np.pi
            c = -c
            s = -s

        G = np.identity(num_rows)
        G[col, col] = c
        G[row, row] = c
        phi_list.append(phi)
        G[row, col] = s
        G[col, row] = -s

        R[col], R[row] = R[col]*c + R[row]*(-s), R[col]*s + R[row]*c
        Q[:, col], Q[:, row] = Q[:, col]*c + Q[:, row]*(-s), Q[:, col]*s + Q[:, row]*c

        # R = np.dot(G, R)

        # Q = np.dot(Q, G.T)

    return Q, R, phi_list


In [2]:
res_1 = QRGivens_double(A)
res_1

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [3]:
res_1 = QRGivens_double(A)
res_2 = QRGivens_fast(A)
res_1, res_2

(array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 (array([[ 0.20980589,  0.82043761, -0.0746245 , ..., -0.06558471,
           0.01640236,  0.08205914],
         [-0.24553226,  0.25899977,  0.39825544, ...,  0.09252276,
           0.11955814,  0.13044741],
         [ 0.14644746, -0.07582128,  0.78959838, ...,  0.00455284,
           0.12288704,  0.05834521],
         ...,
         [ 0.21221915, -0.00747788, -0.03003547, ...,  0.68446064,
           0.17212028,  0.03055006],
         [ 0.14643829,  0.03204696, -0.09706833, ..., -0.40784145,
           0.66767357,  0.06134376],
         [ 0.13328314, -0.08787136, -0.08608019, ..., -0.0753733 ,
          -0.02472106,  0.53696022]]),
  array([[ 3.29541019e+00, -1.24753052e+01,  7.88175248e+00, ...,
           1.04330110e+01,  6.94490219e+00,  7.31857352e+00],
         [ 6.95370715e-17,  2.20402260e+00, -1.36030638e+00, ...,
          -1.70842986e+00, -1.05661688e+00, -1.27465826e+00],
     

In [1]:
from utils import QRGivens, Givens2Matrix
from python_example import QRGivens_double, Givens2Matrix_double

dim = 50

from sklearn.datasets import make_spd_matrix
A = make_spd_matrix(dim)


In [2]:
phi_list = QRGivens_double(A)
%timeit Givens2Matrix_double(phi_list)
%timeit Givens2Matrix(phi_list)

92.5 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
23.3 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:

%timeit QRGivens_double(A)
%timeit QRGivens_fast(A)
%timeit np.linalg.qr(A)
%timeit np.linalg.eigh(A)
%timeit QRGivens_slow(A)

In [10]:

%timeit np.linalg.eigh(A)

17.2 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
